# Samhliðamálheild fyrir ensku og íslensku - ParIce
Hér er unnið með ParIce málheildina. Gangasafnið lesið, prófunargögn síuð út og sett í staðlað form. 

Gert er ráð fyrir að eftirfarandi gögn séu til staðar
- ParIce eins og það er sótt á málföng, `parice_dir`
- Prófunargögn eins og fengin frá Steinþóri, `test_dir`

Eftir að hafa keyrt reikniritið verður `target_dir` eftirfarandi
- `target_dir/x.(is|en)`, þar sem `x` er undirmálheild úr ParIce.
- `target_dir/train.(is|en)` allar málheildir úr ParIce sameinaðar og prófunar- og fínpússunargögn fjarlægð.
- `target_dir/dev.(is|en)` 2000 setningapör valin af handahófi úr sameinuðu ParIce með prófunargögn fjarlægð.
- `target_dir/test-(ees|ema|opensubtitles).(is|en)` Prófunargögn eins og fengin frá Steinþóri. 

In [40]:
# The location of Parice to read
parice_dir = '/work/haukurpj/data/raw/Parice1.0'
# The location of the Parice test data (zipped, provided separatly).
test_set_zip = '/work/haukurpj/data/raw/prófunarsett-20191205T064220Z-001.zip'

# The location of where to write the results
target_dir = '/work/haukurpj/data/formatted/Parice1.0'

from glob import glob
from pprint import pprint

In [41]:
import pathlib

parice_raw_dir = pathlib.Path(parice_dir)
parice_formatted_dir = pathlib.Path(target_dir)
parice_test_zip = pathlib.Path(test_set_zip)

In [42]:
assert parice_raw_dir.exists()
assert parice_test_zip.exists()
if not parice_formatted_dir.exists():
    parice_formatted_dir.mkdir()

## Sækja opensubtitles
Opensubtitles er ekki deilt með ParIce, út af leyfismálum, svo við þurfum að sækja þau gögn og keyra scriptu til þess að para setningar.

In [11]:
!wget -P {parice_raw_dir}/opensubtitles/en.zip http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/raw/en.zip
!wget -P {parice_raw_dir}/opensubtitles/is.zip http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/raw/is.zip

--2020-01-17 09:58:51--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/raw/en.zip
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/raw/en.zip [following]
--2020-01-17 09:58:52--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/raw/en.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19, 86.50.254.18
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13736002649 (13G) [application/zip]
Saving to: ‘/work/haukurpj/data/formatted/Parice1.0/download.php?f=OpenSubtitles%2Fv2018%2Fraw%2Fen.zip’

download.php?f=Open 100%[===================>]  12.79G  7.51MB/s    in 26m 1s  

2020-01-17 10:24:53 (8.39 MB/s) - ‘/work/haukurpj/data/formatted/Parice1.0/download.php?f=OpenSubtitles%2Fv20

In [30]:
!mkdir -p {parice_raw_dir}/opensubtitles/en
!mkdir -p {parice_raw_dir}/opensubtitles/is

In [ ]:
!unzip {parice_raw_dir}/en.zip -d {parice_raw_dir}/en
!unzip {parice_raw_dir}/is.zip -d {parice_raw_dir}/is

### Keyra scriptu til þess að búa til opensubtitles.tmx
Núna þarf að keyra scriptu sem býr til samhliða gögnin fyrir opensubtitles. Til þess þurfum við vísa í íslensku og ensku gögnin og scriptan finnur sjálf `links_to_opus`. Skrárnar þar eru ekki þær nýjustu, þar sem Opus hefur uppfært gögnin sín. Ég (Haukur) bað Starkað (Árnastofnun) um uppfærðar skrár og setti þær í staðinn.

In [4]:
%cd {parice_raw_dir}/opensubtitles
!python read_opus.py --input {parice_raw_dir}/opensubtitles --output {parice_raw_dir}/tmx

/work/haukurpj/data/raw/Parice1.0/opensubtitles
21079 1931
22286 1931
26752 1935
29583 1937
29843 1938
31679 1939
32138 1939
32910 1940
33563 1941
33870 1941
34492 1942
34583 1942
38559 1946
39628 1947
40525 1947
40506 1948
40897 1948
42332 1950
42895 1950
43014 1950
43274 1951
43887 1951
44079 1951
44509 1952
44706 1952
45891 1953
47677 1953
46816 1954
46874 1954
47264 1954
47296 1954
47573 1954
47969 1955
48028 1955
48280 1955
48342 1955
48545 1955
48728 1955
49169 1956
49408 1956
49730 1956
49849 1956
50086 1957
52278 1958
52357 1958
52561 1959
52993 1959
53125 1959
53221 1959
54215 1960
54387 1960
54698 1961
55254 1961
56062 1962
56217 1962
56592 1962
57193 1963
57197 1963
58283 1964
58777 1964
59309 1965
59885 1965
60086 1966
60095 1966
60748 1966
61089 1966
61452 1967
61578 1967
61619 1967
62477 1967
63803 1967
62994 1968
64115 1969
64276 1969
64418 1969
64505 1969
65421 1970
65528 1970
65547 1970
65988 1970
66049 1970
66206 1970
66301 1970
66434 1971
66831 1971
66921 1971
67128 

952640 2007
954439 2007
959403 2007
968992 2007
979594 2007
979595 2007
979596 2007
979597 2007
988050 2007
992243 2007
992244 2007
997062 2007
998660 2007
1010048 2008
1013542 2008
1023481 2008
1024255 2008
1032755 2008
1032821 2008
1034303 2008
1035730 2008
1046163 2008
1059786 2008
1060277 2008
1082868 2008
1099212 2008
1131749 2008
1151915 2008
1157605 2008
1172206 2008
1172570 2008
1205489 2008
1233576 2008
1245734 2008
200465 2008
367882 2008
369436 2008
371746 2008
397892 2008
411477 2008
421715 2008
430922 2008
443274 2008
451079 2008
455538 2008
467197 2008
479341 2008
790686 2008
800080 2008
800241 2008
800308 2008
808435 2008
822832 2008
823671 2008
830515 2008
862846 2008
866439 2008
910970 2008
914798 2008
929425 2008
930072 2008
936501 2008
942385 2008
947802 2008
959337 2008
960731 2008
963743 2008
970416 2008
985699 2008
995039 2008
1022603 2009
1032815 2009
1049413 2009
1055369 2009
1075417 2009
1078588 2009
1078940 2009
1093908 2009
1111422 2009
1114677 2009
1114740 2

In [43]:
!ls -lh {parice_raw_dir}/tmx

total 1.1G
-rwxr-xr-x 1 haukurpj local-staff 4.7M Jan 16 14:01 baekur.tmx
-rwxr-xr-x 1 haukurpj local-staff  16M Jan 16 14:01 biblian.tmx
-rwxr-xr-x 1 haukurpj local-staff 583M Jan 16 14:01 ees.tmx
-rwxr-xr-x 1 haukurpj local-staff 124M Jan 24 11:10 ema.tmx
-rwxr-xr-x 1 haukurpj local-staff 5.3M Jan 16 14:01 eso.tmx
-rwxr-xr-x 1 haukurpj local-staff 5.9M Jan 16 14:01 fornritin.tmx
-rwxr-xr-x 1 haukurpj local-staff 1.2M Jan 24 11:10 hagstofan.tmx
-rwxr-xr-x 1 haukurpj local-staff 8.7M Jan 16 14:01 kde4.tmx
-rw-r--r-- 1 haukurpj local-staff 299M Jan 24 10:16 opensubtitles.tmx
-rwxr-xr-x 1 haukurpj local-staff 1.5M Jan 16 14:01 tatoeba.tmx
-rwxr-xr-x 1 haukurpj local-staff 1.7M Jan 16 14:01 ubuntu.tmx


In [44]:
!wc -l {parice_raw_dir}/tmx/*.tmx

     62087 /work/haukurpj/data/raw/Parice1.0/tmx/baekur.tmx
    260972 /work/haukurpj/data/raw/Parice1.0/tmx/biblian.tmx
   8505867 /work/haukurpj/data/raw/Parice1.0/tmx/ees.tmx
   1617339 /work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx
     63172 /work/haukurpj/data/raw/Parice1.0/tmx/eso.tmx
     87992 /work/haukurpj/data/raw/Parice1.0/tmx/fornritin.tmx
     11448 /work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx
    199643 /work/haukurpj/data/raw/Parice1.0/tmx/kde4.tmx
   6523036 /work/haukurpj/data/raw/Parice1.0/tmx/opensubtitles.tmx
     33059 /work/haukurpj/data/raw/Parice1.0/tmx/tatoeba.tmx
     42295 /work/haukurpj/data/raw/Parice1.0/tmx/ubuntu.tmx
  17406910 total


## Lesa .tmx skrár
Fyrst þarf þó aðeins að lagfæra skrárnar svo þær séu allar með sama tungumála merki.

In [16]:
!grep -m 2 "lang=\"is\"" {parice_raw_dir}/tmx/*.tmx
!grep -m 2 "lang=\"en\"" {parice_raw_dir}/tmx/*.tmx

/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx:  <header adminlang="EN-US" creationdate="20170426T083707Z" creationtool="Moses-to-TMX-converer" creationtoolversion="1.0" datatype="plaintext" o-tmf="Moses plain text files" segtype="sentence" srclang="en">
/work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx:  <header adminlang="en" creationdate="20170319T213035Z" creationtool="mALIGNa" creationtoolversion="2" datatype="plaintext" o-tmf="al" segtype="block" srclang="en">


In [17]:
!sed -i 's/xml:lang="is"/xml:lang="IS-IS"/g' {parice_raw_dir}/tmx/hagstofan.tmx
!sed -i 's/xml:lang="en"/xml:lang="EN-GB"/g' {parice_raw_dir}/tmx/hagstofan.tmx
!sed -i 's/xml:lang="is"/xml:lang="IS-IS"/g' {parice_raw_dir}/tmx/ema.tmx
!sed -i 's/xml:lang="en"/xml:lang="EN-GB"/g' {parice_raw_dir}/tmx/ema.tmx

In [18]:
!grep -m 2 "lang=\"is\"" {parice_raw_dir}/tmx/*.tmx
!grep -m 2 "lang=\"en\"" {parice_raw_dir}/tmx/*.tmx

/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx:  <header adminlang="EN-US" creationdate="20170426T083707Z" creationtool="Moses-to-TMX-converer" creationtoolversion="1.0" datatype="plaintext" o-tmf="Moses plain text files" segtype="sentence" srclang="en">
/work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx:  <header adminlang="en" creationdate="20170319T213035Z" creationtool="mALIGNa" creationtoolversion="2" datatype="plaintext" o-tmf="al" segtype="block" srclang="en">


In [45]:
tmx_files = glob(f'{parice_raw_dir}/tmx/*.tmx')
tmx_files = [pathlib.Path(tmx_file) for tmx_file in tmx_files]
pprint(tmx_files)

[PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/ema.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/biblian.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/fornritin.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/opensubtitles.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/ees.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/tatoeba.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/baekur.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/kde4.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/hagstofan.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/eso.tmx'),
 PosixPath('/work/haukurpj/data/raw/Parice1.0/tmx/ubuntu.tmx')]


In [46]:
import frontend.bulk as b

parice_tmx = b.tmx_split(tmx_files, 'EN-GB', 'IS-IS')
en_parice, is_parice = zip(*parice_tmx)

In [47]:
for corpus in en_parice + is_parice:
    !mv {corpus} {parice_formatted_dir}

In [48]:
corpora = [corpus.stem for corpus in tmx_files]
corpora.sort()
corpora

['baekur',
 'biblian',
 'ees',
 'ema',
 'eso',
 'fornritin',
 'hagstofan',
 'kde4',
 'opensubtitles',
 'tatoeba',
 'ubuntu']

In [49]:
parice_en_list = [f'{parice_formatted_dir}/{corpus}.en' for corpus in corpora]
parice_is_list = [f'{parice_formatted_dir}/{corpus}.is' for corpus in corpora]
pprint(parice_en_list)
pprint(parice_is_list)

['/work/haukurpj/data/formatted/Parice1.0/baekur.en',
 '/work/haukurpj/data/formatted/Parice1.0/biblian.en',
 '/work/haukurpj/data/formatted/Parice1.0/ees.en',
 '/work/haukurpj/data/formatted/Parice1.0/ema.en',
 '/work/haukurpj/data/formatted/Parice1.0/eso.en',
 '/work/haukurpj/data/formatted/Parice1.0/fornritin.en',
 '/work/haukurpj/data/formatted/Parice1.0/hagstofan.en',
 '/work/haukurpj/data/formatted/Parice1.0/kde4.en',
 '/work/haukurpj/data/formatted/Parice1.0/opensubtitles.en',
 '/work/haukurpj/data/formatted/Parice1.0/tatoeba.en',
 '/work/haukurpj/data/formatted/Parice1.0/ubuntu.en']
['/work/haukurpj/data/formatted/Parice1.0/baekur.is',
 '/work/haukurpj/data/formatted/Parice1.0/biblian.is',
 '/work/haukurpj/data/formatted/Parice1.0/ees.is',
 '/work/haukurpj/data/formatted/Parice1.0/ema.is',
 '/work/haukurpj/data/formatted/Parice1.0/eso.is',
 '/work/haukurpj/data/formatted/Parice1.0/fornritin.is',
 '/work/haukurpj/data/formatted/Parice1.0/hagstofan.is',
 '/work/haukurpj/data/form

## Sía út prófunargögn
Núna síum við út prófunargögnin úr ParIce. Byrjum á því að afpakka.

In [50]:
!unzip {parice_test_zip} -d {parice_formatted_dir}

Archive:  /work/haukurpj/data/raw/prófunarsett-20191205T064220Z-001.zip
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/LESTU-MIG.docx  
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/ees.csv  
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/opensubtitles.csv  
  inflating: /work/haukurpj/data/formatted/Parice1.0/pr?funarsett/ema.csv  


In [51]:
!ls {parice_formatted_dir}/prófunarsett

LESTU-MIG.docx	ees.csv  ema.csv  opensubtitles.csv


In [52]:
def read_csv(p_in):
    en_text = []
    is_text = []
    with p_in.open() as f_in:
        for index, line in enumerate(f_in):
            if '### SKJAL ID' in line:
                continue
            if line == '\n':
                continue
            id, en_sent, is_sent = line.split('\t')
            en_text.append(en_sent+'\n')
            is_text.append(is_sent)
    return en_text, is_text

In [53]:
ees_en, ees_is = read_csv(parice_formatted_dir.joinpath('prófunarsett').joinpath('ees.csv'))
ema_en, ema_is = read_csv(parice_formatted_dir.joinpath('prófunarsett').joinpath('ema.csv'))
opensubtitles_en, opensubtitles_is = read_csv(parice_formatted_dir.joinpath('prófunarsett').joinpath('opensubtitles.csv'))
parice_test_is = ees_is + ema_is + opensubtitles_is
parice_test_en = ees_en + ema_en + opensubtitles_en

In [54]:
parice_is = []
for corpus in parice_is_list:
    with open(corpus) as f_in:
        parice_is += f_in.readlines()

parice_en = []
for corpus in parice_en_list:
    with open(corpus) as f_in:
        parice_en += f_in.readlines()

In [55]:
pprint(len(parice_is))
pprint(len(parice_en))
for i in range(2):
    print(parice_is[i], parice_en[i])

3589030
3589030
FYRRI ÞÁTTUR
 BOOK I

I. Veizlan í nýju höllinni.
 CHAPTER I A FETE AT THE NEW PALACE



In [56]:
def get_indices(corpus, sentences):
    indices = []
    for index, line in enumerate(corpus):
        if line in sentences:
            indices.append(index)
    return indices

In [57]:
indicies_test_en = get_indices(parice_en, frozenset(parice_test_en))
indicies_test_is = get_indices(parice_is, frozenset(parice_test_is))
print(len(indicies_test_en), len(indicies_test_is))

193432 207799


Núna þurfum við að fjarlægja setningar úr ParIce sem eru í prófunargögnunum.

In [58]:
indicies_test_intersection = set(indicies_test_is).intersection(set(indicies_test_en))
indicies_test_union = set(indicies_test_is).union(set(indicies_test_en))
print(len(indicies_test_intersection))
print(len(indicies_test_union))
print(len(parice_test_en), len(parice_test_is))

153345
247886
7645 7645


Við notum sammengið, til þess að losa okkur við allan efa um að þýðingarvélin hafi séð þessar setningar áður og einnig til þess að vera með sambærilega vinnslu og Miðeind.

In [59]:
def drop_indices(corpus, indices):
    filtered_corpus = []
    indices = list(indices)
    indices.sort()
    current_index = 0
    for corpus_index, line in enumerate(corpus):
        if current_index == len(indices):
            filtered_corpus.append(line)
        elif indices[current_index] == corpus_index:
            current_index += 1
        else:
            filtered_corpus.append(line)
    return filtered_corpus

In [60]:
drop = indicies_test_union
parice_no_test_en = drop_indices(parice_en, drop)
parice_no_test_is = drop_indices(parice_is, drop)

## Skipta í dev/train
Við skiptum restini af gögnunum í tvennt, dev/train. Við veljum 2000 setningapör í dev og fjarlægjum þær úr restinni. Sá afgangur verður train. Það er hugsanlegt að við veljum tvítekningar í dev en við verðum bara að lifa með það.

In [61]:
seed = 10

import random
random.seed(seed)
parice_dev_en = list(random.sample(parice_no_test_en, 2000))
indieces_dev_en = get_indices(parice_no_test_en, frozenset(parice_dev_en))
random.seed(seed)
parice_dev_is = list(random.sample(parice_no_test_is, 2000))
indieces_dev_is = get_indices(parice_no_test_is, frozenset(parice_dev_is))

In [62]:
print(len(indieces_dev_en), len(indieces_dev_is))

73696 73345


Hægt er að keyra kóðann (illa skrifaður) fyrir neðan til þess að athuga hvort niðurstöðurnar eru réttar. Þær virðast vera það.

In [63]:
#for line in parice_dev_en:
#    print(line)
#    for index, p_line in enumerate(parice_no_test_en):
#        if p_line == line:
#            print(index, p_line)
#            if index not in indieces_dev_en:
#                raise RuntimeError("Error! Not found")

In [64]:
indicies_dev_intersection = set(indieces_dev_en).intersection(set(indieces_dev_is))
indicies_dev_union = set(indieces_dev_en).union(set(indieces_dev_is))
print(len(indicies_dev_intersection))
print(len(indicies_dev_union))
print(len(parice_dev_en), len(parice_dev_is))

62842
84199
2000 2000


In [65]:
for i in range(2):
    print(parice_dev_en[i], parice_dev_is[i])

Slide over, honey.
 Færðu þig, ástin.

• Where the Commission ascertains that a notified body does not meet or no longer meets the requirements for its notification, it shall adopt an implementing act requesting the notifying Member State to take the necessary corrective measures, including withdrawal of notification if necessary.
 • Ef framkvæmdastjórnin kemst að raun um að tilkynnt stofa uppfylli ekki, eða uppfylli ekki lengur, kröfurnar viðvíkjandi tilkynningu hennar, skal hún samþykkja framkvæmdargerð þar sem óskað er eftir því að tilkynningaraðildarríkið geri nauðsynlegar ráðstafanir til úrbóta, þ.m.t. að afturkalla tilkynningu, ef nauðsyn krefur.



In [66]:
drop = indicies_dev_union
parice_train_en = drop_indices(parice_no_test_en, drop)
parice_train_is = drop_indices(parice_no_test_is, drop)

Athugum hvort línufjöldinn gangi ekki upp.

In [67]:
print(len(parice_train_en), len(parice_train_is))
print(len(parice_en) - (len(parice_train_en) + len(indicies_dev_union) + len(indicies_test_union)))

3256945 3256945
0


## Skrifa gögn og hreinsa til
Núna skrifum við gögnin niður í skrár og fjarlægjum tímabundnar skrár.

In [68]:
!ls {target_dir}

baekur.en   dev.is  eso.en	  hagstofan.is	    prófunarsett  ubuntu.en
baekur.is   ees.en  eso.is	  kde4.en	    tatoeba.en	  ubuntu.is
biblian.en  ees.is  fornritin.en  kde4.is	    tatoeba.is
biblian.is  ema.en  fornritin.is  opensubtitles.en  train.en
dev.en	    ema.is  hagstofan.en  opensubtitles.is  train.is


In [69]:
def write_to_file(corpus, path):
    with path.open('w+') as f_in:
        for line in corpus:
            f_in.write(line)
    !wc -l {path}

In [70]:
write_to_file(parice_train_en, parice_formatted_dir.joinpath('train.en'))
write_to_file(parice_train_is, parice_formatted_dir.joinpath('train.is'))
write_to_file(parice_dev_en, parice_formatted_dir.joinpath('dev.en'))
write_to_file(parice_dev_is, parice_formatted_dir.joinpath('dev.is'))
write_to_file(ees_en, parice_formatted_dir.joinpath('test-ees.en'))
write_to_file(ees_is, parice_formatted_dir.joinpath('test-ees.is'))
write_to_file(ema_en, parice_formatted_dir.joinpath('test-ema.en'))
write_to_file(ema_is, parice_formatted_dir.joinpath('test-ema.is'))
write_to_file(opensubtitles_en, parice_formatted_dir.joinpath('test-opensubtitles.en'))
write_to_file(opensubtitles_is, parice_formatted_dir.joinpath('test-opensubtitles.is'))

3256945 /work/haukurpj/data/formatted/Parice1.0/train.en
3256945 /work/haukurpj/data/formatted/Parice1.0/train.is
2000 /work/haukurpj/data/formatted/Parice1.0/dev.en
2000 /work/haukurpj/data/formatted/Parice1.0/dev.is
3245 /work/haukurpj/data/formatted/Parice1.0/test-ees.en
3245 /work/haukurpj/data/formatted/Parice1.0/test-ees.is
2200 /work/haukurpj/data/formatted/Parice1.0/test-ema.en
2200 /work/haukurpj/data/formatted/Parice1.0/test-ema.is
2200 /work/haukurpj/data/formatted/Parice1.0/test-opensubtitles.en
2200 /work/haukurpj/data/formatted/Parice1.0/test-opensubtitles.is


In [71]:
!rm -rf {parice_formatted_dir}/prófunarsett

In [72]:
!ls {target_dir}

baekur.en   ees.is	  hagstofan.en	    tatoeba.is		   train.en
baekur.is   ema.en	  hagstofan.is	    test-ees.en		   train.is
biblian.en  ema.is	  kde4.en	    test-ees.is		   ubuntu.en
biblian.is  eso.en	  kde4.is	    test-ema.en		   ubuntu.is
dev.en	    eso.is	  opensubtitles.en  test-ema.is
dev.is	    fornritin.en  opensubtitles.is  test-opensubtitles.en
ees.en	    fornritin.is  tatoeba.en	    test-opensubtitles.is
